## 8.build_simple_chain
RAG 기반 chain 생성

<img style="float: right;" src="../img/logo.png" width="120"><br>

<div style="text-align: right"> <b>Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial issue : 2025.11.16 </div>
<div style="text-align: right"> last update : 2025.11.16 </div>

개정 이력  
- `2025.11.16` : 노트북 초기 생성 

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
import pandas as pd
import numpy as np
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from langchain_pinecone import PineconeEmbeddings
from rag_pkg.utils.path import RAW_DATA_PATH, INTERMEDIATE_DATA_PATH, PROCESSED_DATA_PATH, PROMPT_CONFIG_PATH
from rag_pkg.module.preprocess import preprocess_for_rag
from rag_pkg.module.models import get_embedding, get_llm
from rag_pkg.module.vector_db import load_documents, get_vector_store
from rag_pkg.module.prompts import build_qa_prompt, save_prompt, save_fewshot_prompt
from rag_pkg.utils.config_loader import load_yaml
from rag_pkg.utils.rag_utils import format_docs, format_docs_with_meta
from rag_pkg.chains import build_simple_chain
from langchain_core.prompts import load_prompt
from langchain_classic.memory import ConversationBufferWindowMemory

### 0. LLM embedding 설정

In [3]:
embedding = get_embedding(
    model = "gemini-embedding-001"
)
embedding

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f6c4cf33620>, async_client=None, model='models/gemini-embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, base_url=None, transport=None, request_options=None)

In [4]:
llm = get_llm(
    model = "gemini-2.5-flash-lite",
    temperature=0.1
)
llm

GoogleGenerativeAI(callbacks=[], model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.1, client=ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.1, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f6c4cf5a120>, default_metadata=(), model_kwargs={}))

### 1. 프롬프트 로드

In [5]:
prompt = load_prompt(PROMPT_CONFIG_PATH / "qa_prompt.yaml")
print(prompt.template)

 You are a whisky expert and sommelier specializing in personalized whisky recommendations. Use the following pieces of retrieved context and chat history to answer the user's question about whisky.
IMPORTANT GUIDELINES: - Always cite the source by including the review link from the context (e.g., [출처: link]) - For each whisky recommendation, clearly mention: * Product name (위스키 이름) * Key characteristics including nose, flavor, and finish * Why you recommend this whisky based on the user's preferences - If you don't know the answer, just say you don't know. DON'T make anything up. - Provide detailed and helpful explanations unless the user requests a brief answer. - You MUST answer in Korean (한국어로 답변해야 합니다).
Context: {context}
History: {chat_history}
Question: {question}


### 2. Document 로드

In [6]:
# 데이터 전처리
review_data_path = RAW_DATA_PATH / "whisky_reviews.csv"
reviews = pd.read_csv(review_data_path)
review_processed = preprocess_for_rag(reviews, min_comments=2)
print(review_processed['document_text'].iloc[0])

위스키 이름: Springbank10-year-old
태그: Green-House

향(Nose) [점수: 94.0]: The nose is full of aromatic power. We still have a little touch of solvent.\nLeather, orange peel, star anise.\nPlum, prunes, dates, figs, clementine, passion fruit peel.\nOld dry wood, dust, old book.\nWe have aromas of old rum, Demerara, and almost a little cane sugar.

맛(Taste) [점수: 96.0]: On the palate it is surprisingly fresh and "almost" light.\nFresh apricot, pineapple, passion fruit, guava, papaya.\nIt is very tropical.\nBut the dominant remains woody, with pretty spices.\nCloves, anise, pepper, bitter chocolate, cinnamon, nutmeg. They are all there.\nWe have fresh mint, some dry aromatic herbs.\nA little barbecue charcoal, smoke.

피니쉬(Finish) [점수: 95.0]: Long finish on liquorice, camphor, smoke, ash and barbecue, light peat.\nPepper, cloves, fresh mint.\nIt's long, comforting, it feels like you're at the edge of the fireplace.


In [7]:
# Documents 추출출
documents = load_documents(
    df=review_processed,
    document_text_col="document_text"
)
print(len(documents))
documents[:5]

716


[Document(metadata={'whisky_name': 'Springbank10-year-old', 'link': 'https://www.whiskybase.com/whiskies/whisky/41678/springbank-10-year-old', 'tags': 'Green-House', 'nose_score': 94.0, 'taste_score': 96.0, 'finish_score': 95.0}, page_content='위스키 이름: Springbank10-year-old\n태그: Green-House\n\n향(Nose) [점수: 94.0]: The nose is full of aromatic power. We still have a little touch of solvent.\\nLeather, orange peel, star anise.\\nPlum, prunes, dates, figs, clementine, passion fruit peel.\\nOld dry wood, dust, old book.\\nWe have aromas of old rum, Demerara, and almost a little cane sugar.\n\n맛(Taste) [점수: 96.0]: On the palate it is surprisingly fresh and "almost" light.\\nFresh apricot, pineapple, passion fruit, guava, papaya.\\nIt is very tropical.\\nBut the dominant remains woody, with pretty spices.\\nCloves, anise, pepper, bitter chocolate, cinnamon, nutmeg. They are all there.\\nWe have fresh mint, some dry aromatic herbs.\\nA little barbecue charcoal, smoke.\n\n피니쉬(Finish) [점수: 95.0]:

### 3. Vector store

In [8]:
vectorstore = get_vector_store(
    documents = documents,
    embedding=embedding,
    type = "faiss",
    dimension=3072
)
retriever = vectorstore.as_retriever()

### 4. RAG chain 빌드

In [9]:
memory = ConversationBufferWindowMemory(return_messages=True, k=5, memory_key="chat_history")

def load_memory(_):  # 무조건 입력을 넣어야... 규칙임
    return memory.load_memory_variables({})["chat_history"]

/tmp/ipykernel_90654/3651531737.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(return_messages=True, k=5, memory_key="chat_history")


In [13]:
chain = build_simple_chain(
    retriever=retriever,
    prompt=prompt,
    llm=llm,
    load_memory_func=load_memory,
    format_docs_func=format_docs_with_meta
)

In [14]:
chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6bf1d32750>, search_kwargs={})
           | RunnableLambda(format_docs_with_meta),
  question: RunnablePassthrough(),
  chat_history: RunnableLambda(load_memory)
}
| PromptTemplate(input_variables=['chat_history', 'context', 'question'], input_types={}, partial_variables={}, template=" You are a whisky expert and sommelier specializing in personalized whisky recommendations. Use the following pieces of retrieved context and chat history to answer the user's question about whisky.\nIMPORTANT GUIDELINES: - Always cite the source by including the review link from the context (e.g., [출처: link]) - For each whisky recommendation, clearly mention: * Product name (위스키 이름) * Key characteristics including nose, flavor, and finish * Why you recommend this whisky based on the user's preferences - If you don't know the answer, just say you don't k

In [15]:
chain.get_graph().print_ascii()

                            +----------------------------------------------+                        
                            | Parallel<context,question,chat_history>Input |                        
                            +----------------------------------------------+                        
                             *******                *               *******                         
                       ******                       *                      ******                   
                   ****                             *                            ******             
+----------------------+                            *                                  ****         
| VectorStoreRetriever |                            *                                     *         
+----------------------+                            *                                     *         
            *                                       *                                     *

### 5. Test

In [19]:
content = chain.invoke("헤비한 육향이나 거칠고 진득한 스모크 계열을 선호해요. 어디 하나 빠지는 거 없이 꽉 찬 스타일이면 더 좋겠어요.")
memory.save_context({"input": "헤비한 육향이나 거칠고 진득한 스모크 계열을 선호해요. 어디 하나 빠지는 거 없이 꽉 찬 스타일이면 더 좋겠어요."}, {"output": content})
print(content)

헤비한 육향과 거칠고 진득한 스모크 계열을 선호하시고, 꽉 찬 스타일을 좋아하시는군요. 이러한 취향에 맞춰 몇 가지 위스키를 추천해 드립니다.

1.  **Brora 5th Release** [출처: https://www.whiskybase.com/whiskies/whisky/299/brora-5th-release]
    *   **향(Nose):** 강렬하고 파워풀한 스모크와 가죽 향이 지배적이며, 멜론, 잘 익은 과일, 바닐라, 호두, 육두구, 후추, 맥아 추출물, 건초와 같은 복합적인 향이 느껴집니다.
    *   **맛(Taste):** 입안을 꽉 채우는 유질감과 크리미함이 특징이며, 스모키함과 함께 호두, 은은한 단맛, 꿀, 시가, 차가운 재, 오래된 책, 자몽의 풍미가 어우러집니다.
    *   **피니쉬(Finish):** 길고 타오르는 듯한 느낌으로, 모닥불 같은 스모크 향이 지속되며 약간의 건조함이 남습니다.
    *   **추천 이유:** "Leathery", "Smokey", "Nutty", "Tobacco"와 같은 태그에서 알 수 있듯이, 요청하신 헤비한 육향과 진득한 스모크 계열의 특징을 잘 가지고 있습니다. 또한, 높은 점수(향 92.0, 맛 93.0, 피니쉬 91.0)에서 알 수 있듯이 풍부하고 꽉 찬 스타일의 위스키입니다.

2.  **Ardbeg 1974 RWD** [출처: https://www.whiskybase.com/whiskies/whisky/46398/ardbeg-1974-rwd]
    *   **향(Nose):** 타르, 숯, 아스팔트, 바다 소금, 바비큐 베이컨, 요오드 밴디지 향이 강하게 느껴지며, 뒤이어 트로피컬 시트러스 향이 파도처럼 밀려옵니다. 레몬 주스, 라임 제스트, 신선하게 짜낸 키나 껍질의 향도 있습니다. 타는 듯하고 육중하며 기름진 피트와 날카롭고 신선한 시트러스 향의 환상적인 조합입니다.
    *   **맛(Taste):** 생 맥아 보리의 강렬한 노트가 느껴지며, 곡물 설탕, 시트러스가 

In [20]:
content = chain.invoke("앞에 소개한 것 중에서 한가지만 추천해줘요. 그리고 이유도 알려줘요.")
memory.save_context({"input": "앞에 소개한 것 중에서 한가지만 추천해줘요. 그리고 이유도 알려줘요."}, {"output": content})
print(content)

앞서 추천해 드린 위스키들 중에서 한 가지만 추천해 드리자면, **Ardbeg 1974 RWD**를 추천해 드립니다.

*   **위스키 이름:** Ardbeg 1974 RWD
*   **향(Nose):** 타르, 숯, 아스팔트, 바다 소금, 바비큐 베이컨, 요오드 밴디지 향이 강하게 느껴지며, 뒤이어 트로피컬 시트러스 향이 파도처럼 밀려옵니다. 레몬 주스, 라임 제스트, 신선하게 짜낸 키나 껍질의 향도 있습니다. 타는 듯하고 육중하며 기름진 피트와 날카롭고 신선한 시트러스 향의 환상적인 조합입니다. [출처: https://www.whiskybase.com/whiskies/whisky/46398/ardbeg-1974-rwd]
*   **맛(Taste):** 생 맥아 보리의 강렬한 노트가 느껴지며, 곡물 설탕, 시트러스가 가미된 꿀, 묵직하고 곰팡이 같은 노트와 함께 강력하고 기름진 피트가 특징입니다. 처음에는 매우 강렬하여 물 몇 방울을 첨가하는 것이 좋습니다. 시트러스 노트와 맥아 설탕이 어우러져 맥아 시트러스 풍미를 형성하며, 다른 피트 느낌은 부드러워집니다. [출처: https://www.whiskybase.com/whiskies/whisky/46398/ardbeg-1974-rwd]
*   **피니쉬(Finish):** 매우 기름지고 오래 지속되며, 다양한 허브 오일, 방향성 오크 오일, 향신료, 약간의 농장 냄새, 그리고 석회암을 연상시키는 무언가가 느껴집니다. [출처: https://www.whiskybase.com/whiskies/whisky/46398/ardbeg-1974-rwd]
*   **추천 이유:** 고객님께서 선호하시는 "헤비한 육향"과 "거칠고 진득한 스모크 계열"이라는 특징에 가장 부합하는 위스키입니다. 특히 "Tar", "charcoal", "asphalt", "bbq bacon strips", "iodine-soaked bandages"와 같은 향 설명은 고객님의 취향을 만족시킬 강렬하고 독특한 풍미를 기대하게 합니다. 또한, "m

In [21]:
content = chain.invoke("Highland Park 1958과 비슷한 느낌의 위스키를 추천받고 싶어요. 어떤 걸 마셔보면 좋을까요?")
memory.save_context({"input": "Highland Park 1958과 비슷한 느낌의 위스키를 추천받고 싶어요. 어떤 걸 마셔보면 좋을까요?"}, {"output": content})
print(content)

Highland Park 1958과 비슷한 느낌의 위스키를 찾으시는군요. 제공해주신 정보와 이전 대화 내용을 바탕으로 몇 가지 위스키를 추천해 드리겠습니다. Highland Park 1958은 셰리, 과일, 꿀, 그리고 약간의 스모크와 같은 복합적인 풍미를 가진 것으로 보입니다. 이러한 특징을 고려하여 다음과 같은 위스키를 추천합니다.

1.  **Highland Park 1959** [출처: https://www.whiskybase.com/whiskies/whisky/21729/highland-park-1959]
    *   **향(Nose):** 딸기잼, 헤더, 꿀, 신선한 민트 잎, 바닐라 크림, 오렌지, 화이트 초콜릿 등 매우 사랑스럽고 복합적인 향이 특징입니다. 몇 시간 동안 코로 음미하고 싶을 정도로 훌륭한 향을 가지고 있습니다. [출처: https://www.whiskybase.com/whiskies/whisky/21729/highland-park-1959]
    *   **맛(Taste):** 크리미하고 스파이시하며, 티크 오일과 민트 향이 느껴집니다. 베리류, 특히 딸기 맛이 나며, 아주 약한 피트 노트도 있습니다. 시간이 지나면서 OBE(Old Bottle Effect) 노트가 줄어들고 쌉싸름한 다크 초콜릿과 메디시널, 우디한 풍미가 나타나 더욱 훌륭해집니다. [출처: https://www.whiskybase.com/whiskies/whisky/21729/highland-park-1959]
    *   **피니쉬(Finish):** 우디하고 시나몬, 브라인(염수) 향이 느껴지며, 바닐라와 꿀이 더해집니다. 처음에는 쌉싸름하지만 달콤한 셰리 언더톤이 나타나며, 바닐라와 민트 향이 길게 이어집니다. [출처: https://www.whiskybase.com/whiskies/whisky/21729/highland-park-1959]
    *   **추천 이유:** Highland Park 1958과 같은 증류소의 빈티지 제품으로, 유사한 복합